# Dance recognition training - full

In [1]:
### --- Download dataset
import zipfile
from pathlib import Path

dataset_path = Path("/content") / "body-postures"

if not dataset_path.exists():
  # !wget https://github.com/synsense/body-postures/archive/refs/heads/master.zip
  !git clone https://github.com/synsense/body-postures.git

  # with zipfile.ZipFile('NMNISTsmall.zip') as zip_file:
  #     for member in zip_file.namelist():
  #         if not os.path.exists('./' + member):
  #             zip_file.extract(member, './')

### --- Install required packages
# %cd body-postures
!pip install -e body-postures
# %cd -
!pip install torchmetrics

Cloning into 'body-postures'...
remote: Enumerating objects: 600, done.
remote: Counting objects: 100% (600/600), done.
remote: Compressing objects: 100% (583/583), done.
remote: Total 600 (delta 10), reused 600 (delta 10), pack-reused 0
Receiving objects: 100% (600/600), 300.62 KiB | 2.66 MiB/s, done.
Resolving deltas: 100% (10/10), done.
Obtaining file:///content/body-postures
     |████████████████████████████████| 78 kB 2.6 MB/s 
  Created wheel for importRosbag: filename=importRosbag-1.0.3-py3-none-any.whl size=25470 sha256=8e658d92d46767222d302abf2135706e04f64332c8c1292ff6b7f91f2ecf9eee
  Stored in directory: /root/.cache/pip/wheels/9a/3e/7d/a37dd6112f5919fe54f9dde7f47ccb3616b61b0a30ac71394d
Successfully built importRosbag
  Running setup.py develop for body-postures
     |████████████████████████████████| 397 kB 4.2 MB/s 


In [ ]:
### --- Imports
from typing import Optional

from tqdm.auto import tqdm
import torch
import torchmetrics
from torch import nn

# Dataset
from body_postures import BodyPostureFrames

### Settings and hyperparameters

In [ ]:
n_classes = 7
batch_size = 256
learning_rate = 1e-3
weight_decay = 0

## Data inspection

In [ ]:
frame_dataset_training = BodyPostureFrames(
    cache_path='cache/frames_training',
    event_count=3000,
    train=True,
    hot_pixel_filter_freq=60
)
print(len(frame_dataset_training))
frame_dataset_validation = BodyPostureFrames(
    cache_path='cache/frames_val',
    event_count=3000,
    train=False,
    hot_pixel_filter_freq=60
)
print(len(frame_dataset_validation))
train_loader = torch.utils.data.DataLoader(frame_dataset_training, batch_size=batch_size)
val_loader = torch.utils.data.DataLoader(frame_dataset_validation, batch_size=batch_size)

classes = {
    0: "background",
    1: "clap",
    2: "mj",
    3: "salive",
    4: "star",
    5: "wave",
    6: "other",
}

### Model definition

In [ ]:
class ANN(nn.Module):
    def __init__(self, n_classes=10):
        super().__init__()
        
        # The actual network
        self.net = nn.Sequential(
            nn.Conv2d(2, 16, kernel_size=(3, 3), stride=2, padding=1, bias=False),  # 16, 64, 64
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=(2, 2)),  # 16, 32, 32
            nn.Dropout2d(0.1),
            
            nn.Conv2d(16, 32, kernel_size=(3, 3), stride=2, padding=1, bias=False),  # 32, 16, 16
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=(2, 2)),  # 32, 8, 8
            nn.Dropout2d(0.25),
            
            nn.Conv2d(32, 32, kernel_size=(3, 3), stride=1, padding=1, bias=False),  # 64, 4, 4
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=(2, 2)),  # 64, 2, 2
            nn.Flatten(),
            nn.Dropout2d(0.5),
            
            nn.Linear(32*4*4, n_classes, bias=False),
            nn.ReLU(),
        )
        
    def forward(self, x):
        # Reshape data
        x = torch.as_tensor(x, dtype=torch.float32).squeeze(1)
        return self.net(x)

In [ ]:
ann = ANN(n_classes)

## -- Prepare training
# - Loss
criterion = torch.nn.CrossEntropyLoss()

# - Optimizer
optimizer = torch.optim.Adam(ann.parameters(), lr=learning_rate, weight_decay=weight_decay)

# - Confusion matrix
confusion_matrix = torchmetrics.ConfusionMatrix(num_classes=n_classes)

# - Training loop
def train(num_epochs: int=100, validate_every: Optional[None]=5):
    
    # Track losses
    training_losses = []
    val_losses = []
    val_accuracy = []
    
    pbar_epoch = tqdm(range(num_epochs), desc="Epochs")
    pbar_batches_tr = tqdm(train_loader, desc="Batches training")
    pbar_batches_val = tqdm(val_loader, desc="Batches validation")
    for epoch in pbar_epoch:
        losses = []
        for data, labels in pbar_batches_tr:
            # Forward pass
            logits = ann(data)
            # Loss for current batch
            loss = criterion(logits, labels)
            # Backward pass
            loss.backward()
            # Update parameters
            optimizer.step()
            optimizer.zero_grad()
            
            loss = loss.item()
            losses.append(loss)
            pbar_batches_tr.set_postfix(loss=loss)
        
        epoch_loss = sum(losses) / len(losses)
        training_losses.append(epoch_loss)
        pbar_batches_tr.set_postfix(loss=epoch_loss)
        
        # Validation
        if validate_every is not None and epoch % validate_every == 0:
            with torch.no_grad():
                val_data = []
                losses = []
                for data, labels in pbar_batches_val:
                    logits = ann(data)
                    # Loss
                    val_loss = criterion(logits, labels)
                    losses.append(val_loss)
                    # Collect outputs and labels
                    predictions = torch.max(logits, dim=1)[1]
                    val_data.append((predictions, labels))
                
                val_losses.append(sum(losses) / len(losses))
                
                # Calculate accuracy
                confusion_list = [confusion_matrix(pred, label) for pred, label in val_data]
                confusion = torch.stack(confusion_list).sum(0)
                val_accuracy = confusion.trace() / confusion.sum()
                
                pbar_batches_val.set_postfix(loss=val_loss, accuracy=val_accuracy)
                
    
            

In [ ]:
train()

Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Batches training: 0it [00:00, ?it/s]

Batches validation: 0it [00:00, ?it/s]

ZeroDivisionError: ignored